# Homework 3

## Inference on Graphical Models

In the previous homework, you computed the posterior probabilities of either the cook (C), the butler (B), or both, being a murdered given the choice of weapons (K = knife, P = poison). In addition there is the possibility that a third party may have:
- participated in the murder with one or both of the cook or butler, 
- acted alone in the murder,
- or not participated. 

In this exercise you will construct a Directed Bayesian Graphical Model or belief network for the available evidence and perform inference on some of the variables. 

Inspector Markov has continued her investigation. Additionally Dr. Turing has had time to perform laboratory analysis. Turing reports to the Inspector that there is a chance that a toxic substance may have been used to incapacitate the victim before the stabbing. So, there are now two possible weapons:
- knife alone, 
- knife with a poison. 

Given this evidence, Inspector Markov must update her beliefs. Further, she needs to perform inference to determine if there are likely combinations of suspects and weapons. 

As a first step in creating the belief network, import the packages you will need for this analysis.

In [1]:
from pgmpy.models import BayesianModel
from pgmpy.factors.discrete import TabularCPD

The joint probability distribution is:

$$p(B,C,BW,CW,M)$$   
where the letters indicate the following variables;   
$B = $ butler committed the crime, {not murderer, murderer},   
$C = $ cook committed the crime, {not murderer, murderer},    
$BW = $ choice of weapon, {knife, knife with poison}, conditional on butler,  
$CW = $ choice of weapon, {knife, knife with poison}, conditional on cook,   
$M = $ murderer {butler or cook, third party alone, combination of butler or cook and third party}.    

Keeping in mind it is possible the cook, the butler and the third party could be guilty, and that participation in the crime in any capacity constitutes guilt, the distribution can be factorized in the following manner:

$$p(B,C,BW,CW,M) = p(B)\ p(C)\ p(BW\ |\ B)\ p(CW\ |\ C)\ p(M\ |\ BW,CW, C, B)$$  

Now you will need to define the skeleton of the graph. Given the independency relationships of the factorized probability distribution define the skeleton of the model (`m_model`) using the `BayesianModel` function.

>**Hint:** Using paper and pencil make a sketch of the graph before you commit your skeleton structure to code. This structure will be identical to the previous homework.  

>**WARNING:** The distribution and its factorization used for this problem are quite different from previous assignments. 

Your next step to create you model is to define the CDP for each independent variable using the `TabularCDP` function. The prior is that butlers are more likely to be murderers than cooks. The tables for these variables are:    


$p(B)$   

| Case | p |
|---|---|
|B0 | 0.2 |
|B1 | 0.8 |    

$p(C)$   

| Case | p |
|---|---|
|C0 | 0.7 |
|C1 | 0.3 |


Using the above tables define the CDPs. Make sure you use variable names consistent with your model.

Next, define the variables $BW$ and $CW$, the conditional probabilities of choice of weapon given butler or cook. These suspects have difference likely choices of weapons. The conditional probability tables for these variables are:

$$p(BW)$$



| | B0| B1 |
|---|---|---|
|BW0 | 0.5 | 0.9 | 
|BW1 | 0.5 | 0.1 | 


Here;  
BW0 = Butler and knife,   
BW1 = Butler and knife with poison.


$$p(CW)$$

| | C0| C1 |
|---|---|---|
|CW0 | 0.5 | 0.3 | 
|CW1 | 0.5 | 0.7 | 


Here;  
CW0 = Cook and knife,   
CW1 = Cook and knife with poison.

>**Note:** when the butler or cook are not a murderer we have a uniform on uninformative distribution. 

Give the above tables define the CDPs. 

Finally, you must define the probability of the murder which are coded:

- **M0:** Named party (cook or butler or both) with no third unnamed party, 
- **M1:** Only the third unnamed party alone (not cook and not butler), 
- **M2:** Named party (butler or cook or both) and unnamed party together. 

This CDP is conditional on BK, CK, C and B. Since there are three possible guilty parties here are 48 possible states; $N_{cook_or_butler} *  N_{BK} * N_{CK} * N_{M} = 2 * 2 * 2 * 3 = 48$ as shown here:

| | p | p | p | p | p | p | p | p | p | p | p | p |  p | p | p | p |
|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|
|| CW0 | CW0 | CW0 | CW0 | CW0 | CW0 | CW0 | CW0 | CW1 | CW1 | CW1 | CW1 | CW1 | CW1 | CW1 | CW1 |
|| BW0 | BW0 | BW0 | BW0 | BW1 | BW1 | BW1 | BW1 | BW0 | BW0 | BW0 | BW0 | BW1 | BW1 | BW1 | BW1 |
|| C0 | C0 | C1 | C1 | C0 | C0 | C1 | C1| C0 | C0 | C1 | C1| C0 | C0 | C1 | C1 |
|| B0 | B1 | B0 | B1 | B0 | B1 | B0 | B1| B0 | B1 | B0 | B1 | B0 | B1 | B0 | B1|
|M0|0.0 | 0.7 | 0.5 | 0.7 | 0.0 | 0.6 | 0.5 |  0.6 | 0.0 | 0.7 | 0.5 |  0.7 | 0.0 | 0.6 | 0.5 |  0.6|
|M1| 1.0 | 0.2 | 0.1 |  0.1 | 1.0 | 0.3 | 0.2 |  0.2 | 1.0 | 0.2 | 0.1 |  0.1 | 1.0 | 0.3 | 0.2 |  0.2 |
|M2| 0.0 | 0.1 | 0.4 |  0.2 | 0.0 | 0.1 | 0.3 |  0.2 | 0.0 | 0.1 | 0.4 |  0.2 | 0.0 | 0.1 | 0.3 |  0.2 |

Using the above table, define the CPD (sorry about all the typing:). 

To complete your belief network, use the `add_cpds` method. 

> **Hint:** Before going any further make sure you apply the `check_model` method to your complete model. 

Next that the independencies of all the variables in your model are correct using the `local_independencies` method. 

**Question:** Is your graph an I-map of the distribution and why?

ANS: 

Now you are ready to perform inference on your model. 

For the first query determine the marginal probability of weapon choice for the butler (with no evidence) using the variable elimination method. 

Next make query to find the marginal probability of weapon choice with evidence the butler is a murderer.

How are the marginal probabilities change by introducing the evidence? Why do you think this is the case?

ANS: 

Now, turn your attention to the variable M. For your first query on this variable use evidence that the cook and the butler selected knife and poison as the weapon. 

Now perform a query on the M table with evidence that the cook and the butler selected a knife alone as the weapon.

How can you best explain the change in the marginal distributions given the different evidence?

ANS: 

Now, perform a query on the M variable with evidence that the cook was not involved and the butler's choice of weapon was the knife alone.  

For the final set of queries you will use the belief propagation method to your DAG. Perform belief propagation and print the factors.

How many cliques are in the graph and what is the maximum span (number of nodes)? 

ANS: 

Now query the BW and M variables using evidence that the cook is not a murderer.

Compare these marginal distributions to the marginal distributions you obtained with simple variable elimination. Is this result what you expected? 

ANS: 

There are many other possible queries you can perform on this model. You may wish to explore some more of these. 